In [1]:
# Number of forecast days - should be set to 56 days for the Kaggle competition
# epochs should be set to 15 and input_parameters to 100 for an accurate but short run

forecast_days=56
epochs = 15
input_parameter=100

# Initial imports
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import math
import tensorflow as tf
import datetime as dt
from datetime import date,timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from numpy import array

In [2]:
# create lists of Japanese Stock Market Holidays (non-weekend)
Japan_Holidays_2019 = ['2019-01-01','2019-01-02','2019-01-03','2019-01-14','2019-02-11', '2019-03-21','2019-04-29','2019-05-03','2019-05-06','2019-07-15','2019-08-12','2019-09-16','2019-09-23', '2019-10-14','2019-11-04','2019-12-31']
Japan_Holidays_2020 = ['2020-01-01', '2020-01-02', '2020-01-03','2020-01-13','2020-02-11','2020-02-24','2020-03-20','2020-04-29','2020-05-04','2020-05-05','2020-05-06','2020-07-23','2020-07-24','2020-08-10','2020-09-21','2020-09-22','2020-11-03','2020-11-23','2020-12-31']
Japan_Holidays_2021 = ['2021-01-01','2021-01-11','2021-02-11','2021-02-23','2021-04-29','2021-05-03','2021-05-04','2021-05-05','2021-07-22','2021-07-23','2021-08-09','2021-09-20','2021-09-23','2021-11-03','2021-11-23','2021-12-31']
Japan_Holidays_2022 = ['2022-01-03','2022-01-10','2022-02-11','2022-02-23','2022-03-21','2022-04-29','2022-05-03','2022-05-04','2022-05-05','2022-07-18','2022-08-11','2022-09-19','2022-09-23','2022-10-10','2022-11-03','2022-11-23']
# concatenate lists
Japan_Holidays = Japan_Holidays_2019+Japan_Holidays_2020+Japan_Holidays_2021+Japan_Holidays_2022
# convert to actual dates 
Japan_Holidays_2019to2022= [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in Japan_Holidays]

In [3]:
# Load the stock prices file
file="./train_files/stock_prices.csv"
stock_df=pd.read_csv(file)
stock_df.tail()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098
2332530,20211203_9997,2021-12-03,9997,690.0,711.0,686.0,696.0,381100,1.0,NaN,False,0.018414


In [4]:
# see how much data there is
stock_df.shape

(2332531, 12)

In [5]:
# find null values
stock_df[stock_df['Close'].isnull()].isnull().sum()

RowId                  0
Date                   0
SecuritiesCode         0
Open                7608
High                7608
Low                 7608
Close               7608
Volume                 0
AdjustmentFactor       0
ExpectedDividend    7584
SupervisionFlag        0
Target               238
dtype: int64

In [6]:
# look at some of the null values
stock_df[stock_df['Close'].isnull()]

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
401,20170104_3540,2017-01-04,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN
1753,20170104_9539,2017-01-04,9539,NaN,NaN,NaN,NaN,0,1.0,NaN,False,-0.004149
2266,20170105_3540,2017-01-05,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN
2511,20170105_4621,2017-01-05,4621,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.000000
4131,20170106_3540,2017-01-06,3540,NaN,NaN,NaN,NaN,0,1.0,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2330563,20211203_1787,2021-12-03,1787,NaN,NaN,NaN,NaN,0,1.0,NaN,False,-0.030351
2330786,20211203_2761,2021-12-03,2761,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.000000
2331453,20211203_5918,2021-12-03,5918,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.015625
2332336,20211203_9083,2021-12-03,9083,NaN,NaN,NaN,NaN,0,1.0,NaN,False,0.009615


In [7]:
# remove null values
stock_df.dropna()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
13269,20170116_2590,2017-01-16,2590,6060.0,6130.0,6060.0,6060.0,125800,1.0,30.0,False,0.005051
13726,20170116_4699,2017-01-16,4699,1119.0,1119.0,1081.0,1087.0,11500,1.0,0.0,False,-0.032833
26179,20170125_1928,2017-01-25,1928,1886.0,1894.5,1865.0,1871.5,3310500,1.0,32.0,False,-0.018336
26249,20170125_2217,2017-01-25,2217,515.0,517.0,512.0,512.0,162000,1.0,4.0,False,0.016000
26281,20170125_2353,2017-01-25,2353,153.0,154.0,151.0,152.0,431700,1.0,0.0,False,-0.006452
...,...,...,...,...,...,...,...,...,...,...,...,...
2320290,20211125_8923,2021-11-25,8923,1047.0,1051.0,1030.0,1030.0,389700,1.0,38.0,False,-0.073099
2320357,20211125_9278,2021-11-25,9278,995.0,995.0,989.0,990.0,12800,1.0,0.0,False,0.000000
2320370,20211125_9369,2021-11-25,9369,2000.0,2009.0,1989.0,1990.0,69100,1.0,23.0,False,-0.057692
2320456,20211125_9717,2021-11-25,9717,1258.0,1260.0,1222.0,1223.0,313200,1.0,50.0,False,-0.127517


In [8]:
stock_df.shape

(2332531, 12)

In [9]:
# create a new df which will only use the "Close" column for predictions
close_df=stock_df[['Date','SecuritiesCode','Close']]
close_df

,Date,SecuritiesCode,Close
0,2017-01-04,1301,2742.0
1,2017-01-04,1332,571.0
2,2017-01-04,1333,3210.0
3,2017-01-04,1376,1550.0
4,2017-01-04,1377,3330.0
...,...,...,...
2332526,2021-12-03,9990,528.0
2332527,2021-12-03,9991,794.0
2332528,2021-12-03,9993,1645.0
2332529,2021-12-03,9994,2389.0


In [10]:
# create a list of all the securities
one_date_df=stock_df.loc[(stock_df['Date']=="2017-01-04"),['SecuritiesCode']]
securities=one_date_df['SecuritiesCode'].tolist()

# for testing can shorten the number of stocks to 10
#del securities[10:]
#securities=[2053,2613,7749,1835,6287]
print(securities)

[1301, 1332, 1333, 1376, 1377, 1379, 1381, 1407, 1414, 1417, 1419, 1429, 1435, 1515, 1518, 1605, 1662, 1663, 1712, 1716, 1719, 1720, 1721, 1723, 1726, 1762, 1766, 1775, 1780, 1787, 1793, 1799, 1801, 1802, 1803, 1805, 1808, 1810, 1811, 1812, 1813, 1814, 1815, 1820, 1821, 1822, 1833, 1835, 1852, 1860, 1861, 1870, 1871, 1873, 1878, 1879, 1882, 1884, 1885, 1888, 1890, 1893, 1898, 1899, 1911, 1914, 1921, 1925, 1926, 1928, 1929, 1930, 1934, 1938, 1939, 1941, 1942, 1944, 1945, 1946, 1949, 1950, 1951, 1952, 1954, 1959, 1961, 1963, 1965, 1967, 1968, 1969, 1973, 1975, 1976, 1979, 1980, 1981, 1982, 2001, 2002, 2003, 2004, 2009, 2053, 2060, 2108, 2109, 2114, 2117, 2120, 2121, 2124, 2127, 2130, 2146, 2148, 2150, 2153, 2154, 2157, 2158, 2160, 2168, 2170, 2175, 2181, 2183, 2185, 2193, 2198, 2201, 2204, 2206, 2207, 2208, 2209, 2211, 2212, 2217, 2220, 2221, 2222, 2226, 2229, 2264, 2266, 2267, 2268, 2269, 2270, 2281, 2282, 2288, 2292, 2294, 2296, 2301, 2305, 2307, 2309, 2315, 2317, 2325, 2326, 2327, 232

In [11]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [],[]
    for i in range(len(dataset)-time_step-1):
        a=dataset[i:(i+time_step),0]
        dataX.append(a)
        dataY.append(dataset[i+time_step,0])
    return np.array(dataX),np.array(dataY)

In [ ]:
# Create the predictions for all stocks

stock_predictions={}
start_prices={}

for stock in securities:
    one_stock_df=close_df.loc[close_df['SecuritiesCode']==stock]
    one_stock_df=one_stock_df.dropna()
    one_stock_df.reset_index(drop=True, inplace=True)
    
    # get the last stock price
    start_price = one_stock_df["Close"].iloc[-1]
    start_prices[stock]=start_price
    
    # convert Date column to datetime 
    one_stock_df['Date'] = pd.to_datetime(one_stock_df['Date']).dt.date
    
    # only use the Close column for predicting
    one_stock_close_df=one_stock_df.reset_index()['Close']
    
    # remove previous model data
    tf.keras.backend.clear_session()
    
    ### LSTM is sensitive to the scale of the data - so we apply MinMax scaler
    scaler=MinMaxScaler(feature_range=(0,1))
    one_stock_scaler=scaler.fit_transform(np.array(one_stock_close_df).reshape(-1,1))
    
    # splitting the dataset into train and test splits
    training_size=int(len(one_stock_scaler)*0.65)
    test_size=len(one_stock_scaler)-training_size
    train_data,test_data=one_stock_scaler[0:training_size,:],one_stock_scaler[training_size:len(one_stock_scaler),:1]
    
    # reshape into X=t,t+1,t+2,t+3 and Y=t+4
    time_step = input_parameter
    X_train, y_train=create_dataset(train_data, time_step)
    X_test, y_test=create_dataset(test_data, time_step)
    
    # reshape input to be [samples, time steps, features] which is required for LSTM
    X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],1)
    

    
    # Create the Stacked LSTM model
    model=Sequential()
    model.add(LSTM(int(input_parameter/2),return_sequences=True,input_shape=(input_parameter,1)))
    model.add(LSTM(int(input_parameter/2),return_sequences=True))
    model.add(LSTM(int(input_parameter/2)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')

    # fit to the model
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=epochs,batch_size=64,verbose=1)
    
    # prediction and check performance metrics
    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)
    
    # transform back to original form
    train_predict=scaler.inverse_transform(train_predict)
    test_predict=scaler.inverse_transform(test_predict)
    
    # setup to predict future stock prices
    input_start=len(test_data)-input_parameter
    x_input=test_data[input_start:].reshape(1,-1)
    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()
    
    # prediction for the number of forecast days 
    lst_output=[]
    n_steps=input_parameter
    i=0

    while(i<forecast_days):
            if(len(temp_input)>input_parameter):
                x_input=np.array(temp_input[1:])
                x_input=x_input.reshape(1,-1)
                x_input=x_input.reshape((1,n_steps,1))
                yhat=model.predict(x_input,verbose=0)
                temp_input.extend(yhat[0].tolist())
                temp_input=temp_input[1:]
                lst_output.extend(yhat.tolist())
                i=i+1
            else:
                x_input=x_input.reshape((1,n_steps,1))
                yhat=model.predict(x_input,verbose=0)
                temp_input.extend(yhat[0].tolist())
                lst_output.extend(yhat.tolist())
                i=i+1
            
    # add the stock id and predictions to a list
    predicted_scaler=scaler.inverse_transform(lst_output)
    predicted_values=predicted_scaler.tolist()
    stock_predictions[stock]=predicted_values
    print(f"Securities code: {stock} just finished.")    
    print("\n")

Epoch 1/15
11/11 [==============================] - 5s 180ms/step - loss: 0.0500 - val_loss: 0.0020
Epoch 2/15
11/11 [==============================] - 1s 97ms/step - loss: 0.0143 - val_loss: 0.0091
Epoch 3/15
11/11 [==============================] - 1s 103ms/step - loss: 0.0075 - val_loss: 9.7519e-04
Epoch 4/15
11/11 [==============================] - 1s 89ms/step - loss: 0.0049 - val_loss: 9.4227e-04
Epoch 5/15
11/11 [==============================] - 1s 103ms/step - loss: 0.0040 - val_loss: 9.9596e-04
Epoch 6/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0037 - val_loss: 0.0010
Epoch 7/15
11/11 [==============================] - 1s 105ms/step - loss: 0.0036 - val_loss: 9.4899e-04
Epoch 8/15
11/11 [==============================] - 1s 96ms/step - loss: 0.0034 - val_loss: 9.2838e-04
Epoch 9/15
11/11 [==============================] - 1s 92ms/step - loss: 0.0033 - val_loss: 8.8058e-04
Epoch 10/15
11/11 [==============================] - 1s 91ms/step - loss: 0.0031 

Epoch 6/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0048 - val_loss: 0.0104
Epoch 7/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0047 - val_loss: 0.0093
Epoch 8/15
11/11 [==============================] - 1s 95ms/step - loss: 0.0046 - val_loss: 0.0081
Epoch 9/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0045 - val_loss: 0.0068
Epoch 10/15
11/11 [==============================] - 1s 95ms/step - loss: 0.0044 - val_loss: 0.0055
Epoch 11/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0044 - val_loss: 0.0062
Epoch 12/15
11/11 [==============================] - 1s 94ms/step - loss: 0.0043 - val_loss: 0.0065
Epoch 13/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0043 - val_loss: 0.0055
Epoch 14/15
11/11 [==============================] - 1s 95ms/step - loss: 0.0042 - val_loss: 0.0050
Epoch 15/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0041 - val_loss: 0.0036
S

11/11 [==============================] - 1s 95ms/step - loss: 0.0014 - val_loss: 0.0054
Epoch 10/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0013 - val_loss: 0.0049
Epoch 11/15
11/11 [==============================] - 1s 95ms/step - loss: 0.0013 - val_loss: 0.0058
Epoch 12/15
11/11 [==============================] - 1s 103ms/step - loss: 0.0012 - val_loss: 0.0072
Epoch 13/15
11/11 [==============================] - 1s 97ms/step - loss: 0.0012 - val_loss: 0.0078
Epoch 14/15
11/11 [==============================] - 1s 104ms/step - loss: 0.0011 - val_loss: 0.0054
Epoch 15/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0011 - val_loss: 0.0072
Securities code: 1419 just finished.


Epoch 1/15
11/11 [==============================] - 5s 179ms/step - loss: 0.0802 - val_loss: 0.0058
Epoch 2/15
11/11 [==============================] - 1s 104ms/step - loss: 0.0183 - val_loss: 0.0100
Epoch 3/15
11/11 [==============================] - 1s 101ms/step - los

11/11 [==============================] - 1s 103ms/step - loss: 0.0042 - val_loss: 0.0021
Epoch 14/15
11/11 [==============================] - 1s 97ms/step - loss: 0.0040 - val_loss: 0.0020
Epoch 15/15
11/11 [==============================] - 1s 103ms/step - loss: 0.0040 - val_loss: 0.0020
Securities code: 1605 just finished.


Epoch 1/15
11/11 [==============================] - 5s 176ms/step - loss: 0.1184 - val_loss: 0.0190
Epoch 2/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0186 - val_loss: 0.0140
Epoch 3/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0125 - val_loss: 0.0090
Epoch 4/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0110 - val_loss: 0.0148
Epoch 5/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0099 - val_loss: 0.0083
Epoch 6/15
11/11 [==============================] - 1s 96ms/step - loss: 0.0093 - val_loss: 0.0067
Epoch 7/15
11/11 [==============================] - 1s 100ms/step - loss: 

Epoch 3/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0099 - val_loss: 0.0085
Epoch 4/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0075 - val_loss: 0.0116
Epoch 5/15
11/11 [==============================] - 1s 95ms/step - loss: 0.0053 - val_loss: 0.0047
Epoch 6/15
11/11 [==============================] - 1s 107ms/step - loss: 0.0042 - val_loss: 0.0037
Epoch 7/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0034 - val_loss: 0.0029
Epoch 8/15
11/11 [==============================] - 1s 105ms/step - loss: 0.0031 - val_loss: 0.0023
Epoch 9/15
11/11 [==============================] - 1s 96ms/step - loss: 0.0030 - val_loss: 0.0022
Epoch 10/15
11/11 [==============================] - 1s 104ms/step - loss: 0.0029 - val_loss: 0.0022
Epoch 11/15
11/11 [==============================] - 1s 105ms/step - loss: 0.0029 - val_loss: 0.0022
Epoch 12/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0029 - val_loss: 0.0022


...

11/11 [==============================] - 1s 101ms/step - loss: 0.0094 - val_loss: 0.0014
Epoch 9/15
11/11 [==============================] - 1s 104ms/step - loss: 0.0090 - val_loss: 4.1747e-04
Epoch 10/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0082 - val_loss: 5.7788e-04
Epoch 11/15
11/11 [==============================] - 1s 109ms/step - loss: 0.0081 - val_loss: 1.8784e-04
Epoch 12/15
11/11 [==============================] - 1s 105ms/step - loss: 0.0075 - val_loss: 4.8093e-04
Epoch 13/15
11/11 [==============================] - 1s 101ms/step - loss: 0.0086 - val_loss: 2.3231e-04
Epoch 14/15
11/11 [==============================] - 1s 106ms/step - loss: 0.0078 - val_loss: 2.4387e-04
Epoch 15/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0067 - val_loss: 2.5962e-04
Securities code: 2471 just finished.


Epoch 1/15
11/11 [==============================] - 5s 183ms/step - loss: 0.0837 - val_loss: 0.0113
Epoch 2/15
11/11 [====================

11/11 [==============================] - 1s 103ms/step - loss: 0.0012 - val_loss: 0.0054
Epoch 13/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0012 - val_loss: 0.0059
Epoch 14/15
11/11 [==============================] - 1s 105ms/step - loss: 0.0011 - val_loss: 0.0047
Epoch 15/15
11/11 [==============================] - 1s 101ms/step - loss: 0.0011 - val_loss: 0.0039
Securities code: 2491 just finished.


Epoch 1/15
11/11 [==============================] - 5s 180ms/step - loss: 0.0863 - val_loss: 0.0024
Epoch 2/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0216 - val_loss: 0.0111
Epoch 3/15
11/11 [==============================] - 1s 96ms/step - loss: 0.0154 - val_loss: 0.0022
Epoch 4/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0114 - val_loss: 0.0026
Epoch 5/15
11/11 [==============================] - 1s 96ms/step - loss: 0.0097 - val_loss: 0.0024
Epoch 6/15
11/11 [==============================] - 1s 99ms/step - loss: 

11/11 [==============================] - 5s 175ms/step - loss: 0.0876 - val_loss: 0.0078
Epoch 2/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0169 - val_loss: 0.0221
Epoch 3/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0111 - val_loss: 0.0086
Epoch 4/15
11/11 [==============================] - 1s 101ms/step - loss: 0.0087 - val_loss: 0.0098
Epoch 5/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0077 - val_loss: 0.0071
Epoch 6/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0070 - val_loss: 0.0047
Epoch 7/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0067 - val_loss: 0.0038
Epoch 8/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0064 - val_loss: 0.0035
Epoch 9/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0062 - val_loss: 0.0032
Epoch 10/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0061 - val_loss: 0.0034
Epoch 11/15
11

11/11 [==============================] - 1s 97ms/step - loss: 0.0041 - val_loss: 0.0050
Epoch 7/15
11/11 [==============================] - 1s 94ms/step - loss: 0.0035 - val_loss: 0.0029
Epoch 8/15
11/11 [==============================] - 1s 97ms/step - loss: 0.0033 - val_loss: 0.0015
Epoch 9/15
11/11 [==============================] - 1s 93ms/step - loss: 0.0032 - val_loss: 0.0015
Epoch 10/15
11/11 [==============================] - 1s 98ms/step - loss: 0.0032 - val_loss: 0.0017
Epoch 11/15
11/11 [==============================] - 1s 93ms/step - loss: 0.0031 - val_loss: 0.0022
Epoch 12/15
11/11 [==============================] - 1s 97ms/step - loss: 0.0031 - val_loss: 0.0021
Epoch 13/15
11/11 [==============================] - 1s 115ms/step - loss: 0.0030 - val_loss: 0.0023
Epoch 14/15
11/11 [==============================] - 1s 111ms/step - loss: 0.0029 - val_loss: 0.0019
Epoch 15/15
11/11 [==============================] - 1s 94ms/step - loss: 0.0027 - val_loss: 0.0022
Securities co

11/11 [==============================] - 1s 98ms/step - loss: 0.0053 - val_loss: 0.0106
Epoch 11/15
11/11 [==============================] - 1s 111ms/step - loss: 0.0056 - val_loss: 0.0090
Epoch 12/15
11/11 [==============================] - 1s 112ms/step - loss: 0.0055 - val_loss: 0.0084
Epoch 13/15
11/11 [==============================] - 1s 100ms/step - loss: 0.0049 - val_loss: 0.0081
Epoch 14/15
11/11 [==============================] - 1s 110ms/step - loss: 0.0043 - val_loss: 0.0079
Epoch 15/15
11/11 [==============================] - 1s 99ms/step - loss: 0.0041 - val_loss: 0.0076
Securities code: 2594 just finished.


Epoch 1/15
11/11 [==============================] - 5s 186ms/step - loss: 0.1936 - val_loss: 0.0183
Epoch 2/15
11/11 [==============================] - 1s 103ms/step - loss: 0.0265 - val_loss: 5.5919e-04
Epoch 3/15
11/11 [==============================] - 1s 102ms/step - loss: 0.0158 - val_loss: 7.0840e-04
Epoch 4/15
11/11 [==============================] - 1s 106ms/

In [ ]:
stock_predictions

In [ ]:
for i in range(len(stock_predictions)):
    for j in range(forecast_days):
        print(stock_predictions[securities[i]][j])

In [ ]:
# calculate the daily percent change for all stocks for all forecast days and put into a series
stock_returns={}

for i in range(len(stock_predictions)):
    percent_returns=[]
    first_return=0
    first_return=(stock_predictions[securities[i]][0][0]-start_prices[securities[i]])/start_prices[securities[i]]
    percent_returns.append(first_return)
    for j in range(forecast_days):
        if j < forecast_days-1:
            next_return= (stock_predictions[securities[i]][j+1][0]-stock_predictions[securities[i]][j][0])/stock_predictions[securities[i]][j][0]
            percent_returns.append(next_return)       
    stock_returns[securities[i]]=percent_returns

In [ ]:
stock_returns

In [ ]:
# create a rankings_df with stock return rankings
total_days=len(one_stock_df)
new_date=one_stock_df['Date'][total_days-1]
column_names =  ['Date', 'SecuritiesCode', 'Rank']
rankings_df = pd.DataFrame(columns=column_names)

for i in range(forecast_days):
    new_date=pd.to_datetime(new_date+dt.timedelta(days=1))
    while new_date.weekday()==5 or new_date.weekday()==6 or new_date.date() in Japan_Holidays_2019to2022:
        new_date=pd.to_datetime(new_date+dt.timedelta(days=1))
        
    daily_percent_change=[]
    for j in range(len(stock_returns)):
        daily_percent_change.append(stock_returns[securities[j]][i])
    
    daily_percent_change_sr=pd.Series(daily_percent_change)
    daily_percent_change_sr.index=securities
    stock_rank = daily_percent_change_sr.rank(method='first',ascending=False)
    stock_rank=stock_rank.astype(int)    
    
    for j in range(len(stock_rank)):
        rankings_df.loc[j+i*len(stock_rank)] = [new_date.date(),securities[j], stock_rank[securities[j]]-1]     
        

In [ ]:
# check the daily percent change
stock_returns[securities[0]]

In [ ]:
# check the rankings_df   
rankings_df.head(20)  

In [ ]:
# write rankings_df to a csv file
filepath="rankings2000s.56d.15e.100ip.csv"
rankings_df.to_csv(filepath,index=False)

In [ ]:
# create a new df with predictions
total_days=len(one_stock_df)
new_date=one_stock_df['Date'][total_days-1]
column_names =  ['Date', 'SecuritiesCode', 'Close']
predictions_df = pd.DataFrame(columns=column_names)


for i in range(forecast_days):
    new_date=pd.to_datetime(new_date+dt.timedelta(days=1))
    while new_date.weekday()==5 or new_date.weekday()==6 or new_date.date() in Japan_Holidays_2019to2022:
        new_date=pd.to_datetime(new_date+dt.timedelta(days=1))
        
    for j in range(len(stock_predictions)):
        predictions_df.loc[j+i*len(stock_predictions)] = [new_date.date(),securities[j], stock_predictions[securities[j]][i][0]]     
        

In [ ]:
stock_predictions

In [ ]:
predictions_df.head(50)

In [ ]:
# write close price predictions_df to a csv file
filepath="predictions2000s.56d.15e.100ip.csv"
predictions_df.to_csv(filepath,index=False)